In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats
import gmaps

import json

# Import API key
from api_keys import g_key2

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Input File (CSV)
input_data_file = "output_data/cities.csv"


In [2]:
weather_df = pd.read_csv(input_data_file)

weather_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness
0,0,-15.1961,12.1522,58.51,58.51,91,Mossamedes,AO,4.18,27
1,0,42.6001,-73.9662,65.16,66.52,76,Albany,US,2.13,100
2,0,-34.5833,150.8552,66.06,66.06,56,Flinders,AU,6.11,4
3,0,71.9167,114.0833,52.38,52.38,80,Saskylakh,RU,14.23,100
4,0,-6.2222,-57.7528,74.17,74.17,70,Jacareacanga,BR,2.59,67


In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key2)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness
14,0,-21.3585,55.7679,73.53,74.21,60,Saint-Philippe,RE,2.42,0
74,0,47.8043,67.7144,71.58,71.58,28,Zhezkazgan,KZ,8.95,0
133,0,-21.3667,55.6167,72.54,73.20,58,Saint-Joseph,RE,3.04,0
135,0,-23.7000,133.8833,78.35,78.35,11,Alice Springs,AU,5.75,0
143,0,29.0331,21.5482,77.32,77.32,56,Jalu,LY,7.29,0
271,0,31.6100,34.7642,74.37,78.08,100,Kiryat Gat,IL,5.99,0
288,0,-2.7906,-49.6717,70.50,70.50,67,Baião,BR,1.63,0
341,0,-20.9373,55.2919,73.13,75.25,47,Le Port,RE,3.24,0
352,0,45.1667,10.7833,69.78,74.44,83,Province of Mantua,IT,5.01,0
392,0,38.2019,20.4314,79.18,79.18,78,Lixouri,GR,2.28,0


In [7]:
hotel_df = filtered_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=" + g_key2 + "&type=lodging&radius=5000&language=en-us"

for index, row in hotel_df.iterrows():
    # API Parameters    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Construct API URL
    url = base_url + "&location={0},{1}".format(lat,lng)
    
    #print("url: " + url)
    
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.post(url).json()
    
    # extract results
    results = response['results']
    
    #print(results)
    
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_name = results[0]['name']
        hotel_zip_city_country = results[0]['plus_code']['compound_code']
        hotel_city = hotel_zip_city_country[1]
        hotel_country = hotel_zip_city_country[2]
        hotel_lat = results[0]['geometry']['location']['lat']
        hotel_lng = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    #print("------------")
    hotel_df = hotel_df.dropna(how='any')
    

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness,Hotel Name
14,0,-21.3585,55.7679,73.53,74.21,60,Saint-Philippe,RE,2.42,0,Le four à pain
133,0,-21.3667,55.6167,72.54,73.20,58,Saint-Joseph,RE,3.04,0,"""Plantation Bed and Breakfast"
135,0,-23.7000,133.8833,78.35,78.35,11,Alice Springs,AU,5.75,0,Desert Palms Alice Springs
271,0,31.6100,34.7642,74.37,78.08,100,Kiryat Gat,IL,5.99,0,OlusHome
288,0,-2.7906,-49.6717,70.50,70.50,67,Baião,BR,1.63,0,Município de Baião
341,0,-20.9373,55.2919,73.13,75.25,47,Le Port,RE,3.24,0,KORBEY D'OR OUEST
352,0,45.1667,10.7833,69.78,74.44,83,Province of Mantua,IT,5.01,0,Hotel la Favorita
392,0,38.2019,20.4314,79.18,79.18,78,Lixouri,GR,2.28,0,Hotel Summery
418,0,40.7668,-72.8521,67.57,70.05,79,Mastic Beach,US,3.44,0,Brushwood House by Smith Point Beach
467,0,45.4106,40.4361,77.07,77.07,68,Kazanskaya,RU,8.34,0,Tikhiy Dvorik


In [9]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))